# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [65]:
from dolfinx import *  # Assuming the required dolfinx modules are imported
from dolfinx import log, default_scalar_type
from dolfinx import fem, mesh, plot

from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver

import pyvista as pv
import pyvista

import numpy as np
import ufl

import logging
from matplotlib import pyplot as plt

from mpi4py import MPI

In [66]:
# Mesh
L = 20.0
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 1, 1]], [10, 4, 4], mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim, )))

2024-07-27 18:53:26.653 ( 185.849s) [main            ]         graphbuild.cpp:356   INFO| Build local part of mesh dual graph
2024-07-27 18:53:26.653 ( 185.849s) [main            ]           ordering.cpp:204   INFO| GPS pseudo-diameter:(16) 159-0
2024-07-27 18:53:26.653 ( 185.849s) [main            ]           Topology.cpp:1330  INFO| Create topology (single cell type)
2024-07-27 18:53:26.653 ( 185.849s) [main            ]           Topology.cpp:1044  INFO| Create topology (generalised)
2024-07-27 18:53:26.653 ( 185.849s) [main            ]                MPI.cpp:164   INFO| Computing communication graph edges (using NBX algorithm). Number of input edges: 1
2024-07-27 18:53:26.653 ( 185.849s) [main            ]                MPI.cpp:235   INFO| Finished graph edge discovery using NBX algorithm. Number of discovered edges 1
2024-07-27 18:53:26.653 ( 185.849s) [main            ]          partition.cpp:233   INFO| Compute ghost indices
2024-07-27 18:53:26.653 ( 185.849s) [main           

We create two python functions for determining the facets to apply boundary conditions to

In [67]:
def left(x):
    return np.isclose(x[0], 0)

def right(x):
    return np.isclose(x[0], L)

fdim = domain.topology.dim - 1

print(f"Domain topology = {domain.topology.dim}")
print(f"fdim = {domain.topology.dim - 1}\n")

left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

print(f"Left faces = {left_facets}")
print(f"Right faces = {right_facets}")

Domain topology = 3
fdim = 2

Left faces = [  2   8  14  20  38  44  50  56  86  92  98 104 110 159 165 226]
Right faces = [458 459 460 461 506 507 508 509 542 543 544 545 546 564 565 573]


2024-07-27 18:53:26.663 ( 185.859s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (3,0)
2024-07-27 18:53:26.663 ( 185.859s) [main            ]topologycomputation.cpp:870   INFO| Computing mesh connectivity 2 - 3 from transpose.
2024-07-27 18:53:26.663 ( 185.859s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (0,0)
2024-07-27 18:53:26.663 ( 185.859s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (2,0)
2024-07-27 18:53:26.663 ( 185.859s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (0,0) - (3,0)
2024-07-27 18:53:26.663 ( 185.859s) [main            ]topologycomputation.cpp:870   INFO| Computing mesh connectivity 0 - 3 from transpose.
2024-07-27 18:53:26.664 ( 185.859s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (3,0) - (0,0)
2024-07-27 18:53:26.664 ( 185.860s) [main            ]topologycomputation.cpp:79

Next, we create a  marker based on these two functions

In [68]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

print(f"marked_facets = {marked_facets}")
print(f"marked_values = {marked_values}")
print(f"sorted_facets = {sorted_facets}")
print(f"facet_tag = {facet_tag.indices}") # = marked_facets

marked_facets = [  2   8  14  20  38  44  50  56  86  92  98 104 110 159 165 226 458 459
 460 461 506 507 508 509 542 543 544 545 546 564 565 573]
marked_values = [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
sorted_facets = [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
facet_tag = [  2   8  14  20  38  44  50  56  86  92  98 104 110 159 165 226 458 459
 460 461 506 507 508 509 542 543 544 545 546 564 565 573]


We then create a function for supplying the boundary condition on the left side, which is fixed.

In [69]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

print(f"Boundary conditions = {u_bc}")

Boundary conditions = [0. 0. 0.]


To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [70]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
right_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(2))

print(f"left_dofs = {left_dofs}")
print(f"right_dofs = {right_dofs}")

bcs = [fem.dirichletbc(u_bc, left_dofs, V), fem.dirichletbc(u_bc, right_dofs, V)]

left_dofs = [  0   2   4   6   9  10  14  17  22  45  47  49  52  54  58  63  65  67
  69  72  76 123 125 127 130 132 136 141 143 145 148 153 155 157 159 162
 166 245 247 249 252 254 258 263 265 267 270 275 277 279 282 287 289 291
 293 296 300 419 421 423 426 431 433 435 438 443 445 447 450 593 595 597
 600 605 607 609 612 771 773 775 778]
right_dofs = [1107 1108 1109 1110 1112 1113 1115 1117 1121 1269 1270 1271 1273 1274
 1277 1281 1282 1283 1284 1286 1289 1413 1414 1415 1417 1418 1421 1425
 1426 1427 1429 1433 1434 1435 1436 1438 1441 1533 1534 1535 1537 1538
 1541 1545 1546 1547 1549 1553 1554 1555 1557 1561 1562 1563 1564 1566
 1569 1621 1622 1623 1625 1629 1630 1631 1633 1637 1638 1639 1641 1669
 1670 1671 1673 1677 1678 1679 1681 1693 1694 1695 1697]


Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`).

In [71]:
B = fem.Constant(domain, default_scalar_type((0, -10, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

Define the test and solution functions on the space $V$

In [72]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [73]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [74]:
# Elasticity parameters
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
# P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [75]:
P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [76]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [77]:
problem = NonlinearProblem(F, u, bcs)

2024-07-27 18:53:26.725 ( 185.921s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (3,0)
2024-07-27 18:53:26.725 ( 185.921s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (3,0) - (2,0)
2024-07-27 18:53:26.725 ( 185.921s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (3,0)
2024-07-27 18:53:26.725 ( 185.921s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (3,0) - (2,0)
2024-07-27 18:53:26.729 ( 185.925s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (3,0)
2024-07-27 18:53:26.729 ( 185.925s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (3,0) - (2,0)


and then create and customize the Newton solver

In [78]:
solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"


2024-07-27 18:53:26.736 ( 185.932s) [main            ]    SparsityPattern.cpp:385   INFO| Column ghost size increased from 0 to 0


We create a function to plot the solution at each time step.

In [79]:
pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("deformation.gif", fps=3)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.functionspace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

2024-07-27 18:53:29.759 ( 188.955s) [main            ]      dofmapbuilder.cpp:166   INFO| Checking required entities per dimension
2024-07-27 18:53:29.759 ( 188.955s) [main            ]      dofmapbuilder.cpp:264   INFO| Cell type:0, dofmap:160x27
2024-07-27 18:53:29.759 ( 188.955s) [main            ]      dofmapbuilder.cpp:320   INFO| Global index computation
2024-07-27 18:53:29.759 ( 188.955s) [main            ]      dofmapbuilder.cpp:637   INFO| Got 4 index_maps
2024-07-27 18:53:29.759 ( 188.955s) [main            ]      dofmapbuilder.cpp:644   INFO| Get global indices


Finally, we solve the problem over several time steps, updating the z-component of the traction

In [80]:
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert (converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    warped.points[:, :] = warped_n.points
    warped.point_data["mag"][:] = magnitude.x.array
    plotter.update_scalar_bar_range([0, 10])
    plotter.write_frame()
plotter.close()

2024-07-27 18:53:29.867 ( 189.063s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 18:53:30.236 ( 189.432s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 18:53:30.494 ( 189.690s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 7.5835e-09 (tol = 1e-08) r (rel) = 5.93709e-11(tol = 1e-08)
2024-07-27 18:53:30.494 ( 189.690s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


Time step 1, Number of iterations 2, Load [ 0.   0.  -1.5]


2024-07-27 18:53:30.681 ( 189.877s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 18:53:31.040 ( 190.236s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 18:53:31.306 ( 190.502s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 1.67947e-14 (tol = 1e-08) r (rel) = 0.829954(tol = 1e-08)
2024-07-27 18:53:31.306 ( 190.502s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.
2024-07-27 18:53:31.435 ( 190.631s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.


Time step 2, Number of iterations 2, Load [ 0.  0. -3.]


2024-07-27 18:53:31.784 ( 190.980s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 18:53:32.038 ( 191.234s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 1.53385e-14 (tol = 1e-08) r (rel) = 0.812451(tol = 1e-08)
2024-07-27 18:53:32.038 ( 191.234s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


...

Time step 7, Number of iterations 2, Load [  0.    0.  -10.5]


2024-07-27 18:53:35.133 ( 194.329s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 18:53:35.518 ( 194.714s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 18:53:35.804 ( 195.000s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 1.59032e-14 (tol = 1e-08) r (rel) = 1.10516(tol = 1e-08)
2024-07-27 18:53:35.804 ( 195.000s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


Time step 8, Number of iterations 2, Load [  0.   0. -12.]


2024-07-27 18:53:35.931 ( 195.127s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 18:53:36.286 ( 195.481s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 18:53:36.556 ( 195.752s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 1.65778e-14 (tol = 1e-08) r (rel) = 1.24191(tol = 1e-08)
2024-07-27 18:53:36.556 ( 195.752s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


Time step 9, Number of iterations 2, Load [  0.    0.  -13.5]
